# Сводные таблицы

Мы уже видели возможности по исследованию отношений в наборе данных,
предоставляемые абстракцией GroupBy. Сводная таблица (pivot table) — схожая
операция, часто встречающаяся в электронных таблицах и других программах,
работающих с табличными данными. Сводная таблица получает на входе простые
данные в виде столбцов и группирует записи в двумерную таблицу, обеспечивающую многомерное представление данных. Различие между сводными таблицами и операцией GroupBy иногда неочевидно. Можно представлять
сводные таблицы как многомерную версию агрегирующей функции GroupBy.
То есть вы выполняете операцию «разбить, применить, объединить», но как разбиение, так и объединение происходят не на одномерном индексе, а на двумерной
координатной сетке.

## Данные для примеров работы со сводными таблицами

Для примеров из этого раздела мы воспользуемся базой данных пассажиров парохода «Титаник», доступной через библиотеку Seaborn

In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.shape

(891, 15)

In [8]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Этот набор данных содержит информацию о каждом пассажире злополучного
рейса, включая пол, возраст, класс, стоимость билета и многое другое.

## Сводные таблицы «вручную»

Чтобы узнать о данных больше, можно начать с группировки пассажиров по полу,
информации о том, выжил ли пассажир, или какой-то их комбинации. Если вы
читали предыдущий раздел, то можете воспользоваться операцией GroupBy. Например, посмотрим на коэффициент выживаемости в зависимости от пола:

In [25]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


Однако хотелось бы заглянуть немного глубже и увидеть распределение выживших по полу и классу. Говоря языком GroupBy, можно было бы идти следующим
путем: сгруппировать по классу и полу, выбрать выживших, применить агрегирующую функцию среднего значения, объединить получившиеся группы, после чего
выполнить операцию unstack иерархического индекса, чтобы обнажить скрытую
многомерность. В виде кода:

In [34]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Это дает нам лучшее представление о том, как пол и класс влияли на выживаемость, но код начинает выглядеть несколько запутанным. Хотя каждый шаг этого
конвейера представляется вполне осмысленным в свете ранее рассмотренных инструментов, такая длинная строка кода не особо удобна для чтения или использования. Двумерный GroupBy встречается настолько часто, что в состав библиотеки
Pandas был включен удобный метод, pivot_table, позволяющий описывать более
кратко данную разновидность многомерного агрегирования.

## Синтаксис сводных таблиц

Вот эквивалентный вышеприведенной операции код, использующий метод pivot_
table объекта DataFrame:

In [35]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Такая запись несравненно удобнее для чтения, чем подход с GroupBy, при том же
результате. Как и можно было ожидать от трансатлантического круиза начала
XX века, судьба благоприятствовала женщинам и первому классу. Женщины из
первого класса выжили практически все (привет, Роуз!), из мужчин третьего
класса выжила только десятая часть (извини, Джек!).

### Многоуровневые сводные таблицы

Группировку в сводных таблицах, как и при операции GroupBy, можно задавать на
нескольких уровнях и с множеством параметров. Например, интересно взглянуть
на возраст в качестве третьего измерения. Разобьем данные на интервалы по возрасту с помощью функции pd.cut:

In [36]:
age = pd.cut(titanic['age'], [0, 18, 80])
age

0      (18.0, 80.0]
1      (18.0, 80.0]
2      (18.0, 80.0]
3      (18.0, 80.0]
4      (18.0, 80.0]
           ...     
886    (18.0, 80.0]
887    (18.0, 80.0]
888             NaN
889    (18.0, 80.0]
890    (18.0, 80.0]
Name: age, Length: 891, dtype: category
Categories (2, interval[int64]): [(0, 18] < (18, 80]]

In [40]:
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

Мы можем применить ту же стратегию при работе со столбцами. Добавим сюда
информацию о стоимости билета, воспользовавшись функцией pd.qcut для автоматического вычисления квантилей:

In [41]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

Результат представляет собой четырехмерную сводную таблицу с иерархическими
индексами, выведенную
в демонстрирующей отношения между значениями сетке.

### Дополнительные параметы сводных таблиц

Полная сигнатура вызова метода pivot_table объектов DataFrame выглядит следующим образом:

In [54]:
# сигнатура вызова в версии 0.181 библиотеки Pandas
# DataFrame.pivot_table(data, values=None, index=None, columns=None,
# aggfunc='mean', fill_value=None, margins=False,
# dropna=True, margins_name='All')

Ключевое слово aggfunc управляет тем, какой тип агрегирования применяется,
по умолчанию это среднее значение. Как и в GroupBy, спецификация агрегирующей функции может быть строкой с одним из нескольких обычных вариантов
('sum', 'mean', 'count', 'min', 'max' и т. д.) или функцией, реализующей агрегирование (np.sum(), min(), sum() и т. п.). Кроме того, агрегирование может быть
задано в виде словаря, связывающего столбец с любым из вышеперечисленных
вариантов:

In [59]:
titanic.pivot_table(index='sex', columns='class', aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Иногда бывает полезно вычислять итоги по каждой группе. Это можно сделать
с помощью ключевого слова margins:

In [61]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838
